In [1]:
from src.utils import *
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
import os
from pathlib import Path
import hydra
from tqdm import tqdm

In [2]:
gpu = None
    
if torch.cuda.is_available() and gpu is not None:
##This may not be necessary outside the notebook
    dev = f"cuda:{gpu}"
else:
    dev = "cpu"

device = torch.device(dev)

In [3]:
log_flag = False

In [4]:
try:
    if dm_dict:
        print("Not reloading datamodule")

except:
    
    dm_dict = {}
    n_components = 0
    print("Initializing datamodule")

Initializing datamodule


In [5]:
ckpt_path = "outputs/AE/AE_CNN_3D/dataset_output/8_channels_spatial_pooling_with_pca_pre_treatment/"


In [6]:
if os.path.isfile(ckpt_path):
    ckpt_list = [ckpt_path]
    
else:
    ckpt_list = list(Path(ckpt_path).rglob('*.ckpt'))


: 

In [10]:
for ckpt_path in tqdm(ckpt_list):

    ckpt_path = str(ckpt_path)
    print(f"model: {ckpt_path}")
    cfg = get_cfg_from_ckpt_path(ckpt_path, pprint = False)

    
    if cfg.datamodule in list(dm_dict.keys()):
        dm = dm_dict[cfg.datamodule]
        train_ssp_tens, test_ssp_tens, _, coords = loading_datamodule(dm)
    
    else:
        print("Inititing datamodule; Generating train and test datasets")
        dm = hydra.utils.call(cfg.datamodule)
        train_ssp_tens, test_ssp_tens, dm, coords = loading_datamodule(dm)
        dm_dict[cfg.datamodule] = dm
        
    try:
        lit_model = load_model(ckpt_path, dm, device, verbose = True)
    except:
        continue

    if log_flag:
        logger = TensorBoardLogger(save_dir=ckpt_path.split("checkpoints")[0])
    else:
        logger = None

    trainer = Trainer(inference_mode = True, logger=logger)

    trainer.test(model=lit_model,datamodule=dm)


  0%|          | 0/35 [00:00<?, ?it/s]

model: outputs/AE/AE_CNN_3D/dataset_output/8_channels_spatial_pooling_with_pca_pre_treatment/basic loss/pred_1_grad_0_max_pos_0.01_max_value_0.01_fft_0.0_weighted_0_inflection_pos_0.0_inflection_value_0.0/depth_pre_treatment_pca_n_components_107/channels_[1, 8, 8]/upsample_mode_trilinear/linear_later_False_lattent_size_50/1_conv_per_layer/padding_reflect/interp_size_25/pooling_Max_on_dim_spatial/final_upsample_upsample_pooling/act_fn_Elu_final_act_fn_Elu/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-11-13_02-04/checkpoints/val_loss=0.01-epoch=00.ckpt
Inititing datamodule; Generating train and test datasets


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/DATASET/envs/o23gauvr/miniconda3/envs/fsc_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /DATASET/envs/o23gauvr/miniconda3/envs/fsc_env/lib/p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for perfor

Testing DataLoader 0:   0%|          | 0/10 [00:00<?, ?it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  10%|█         | 1/10 [00:00<00:04,  1.97it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  20%|██        | 2/10 [00:00<00:03,  2.63it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  30%|███       | 3/10 [00:00<00:02,  3.27it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  40%|████      | 4/10 [00:01<00:01,  3.57it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  50%|█████     | 5/10 [00:01<00:01,  3.88it/s]PCA pre treatment, depth components: 107
PCA pre treatment, depth components: 107
Testing DataLoader 0:  60%|██████    | 6/10 [00:01<00:00,  4.06it/s]PCA pre treatment, depth components: 107

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           ECS RMSE           │      1.0214208364486694      │
│           SSP RMSE           │      17.496681213378906      │
│           fft loss           │     0.08878815919160843      │
│         fft test mse         │     0.08878815919160843      │
│        gradient loss         │    2.724358273553662e-05     │
│      gradient test mse       │    2.724358273553662e-05     │
│     inflection pos loss      │      1.537736177444458       │
│ inflection position test mse │      1.537736177444458       │
│    inflection value loss     │     0.000915715005248785     │
│  inflection value test mse   │     0.000915715005248785     │
│      max position loss       │      0.6129310131072998      │
│    max position test mse     │      0.6129310131072998      │
│        max value loss        │    0.0024102835450321436     │
│     max value test loss      │    0.0024102835450321436     │
│       prediction loss        │    0.0015642443904653192     │
│     prediction test mse      │    0.0015642443904653192     │
│       test_loss_epoch        │    0.0077176569029688835     │
│      weighted mse loss       │    0.0007780850864946842     │
│  weighted mse loss test mse  │    0.0007780850864946842     │
└──────────────────────────────┴──────────────────────────────┘

...

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/DATASET/envs/o23gauvr/miniconda3/envs/fsc_env/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /DATASET/envs/o23gauvr/miniconda3/envs/fsc_env/lib/p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing DataLoader 0:   0%|          | 0/10 [00:00<?, ?it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  10%|█         | 1/10 [00:00<00:03,  2.43it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  20%|██        | 2/10 [00:00<00:02,  3.65it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  30%|███       | 3/10 [00:00<00:01,  4.54it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  40%|████      | 4/10 [00:00<00:01,  5.01it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  50%|█████     | 5/10 [00:00<00:00,  5.49it/s]PCA pre treatment, depth components: 10
PCA pre treatment, depth components: 10
Testing DataLoader 0:  60%|██████    | 6/10 [00:01<00:00,  5.72it/s]PCA pre treatment, depth components: 10
PCA pre trea

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           ECS RMSE           │             0.0              │
│           SSP RMSE           │      5.993666172027588       │
│           fft loss           │     0.017632270231842995     │
│         fft test mse         │     0.017632270231842995     │
│        gradient loss         │    6.880704006562155e-09     │
│      gradient test mse       │    6.880704006562155e-09     │
│     inflection pos loss      │             0.0              │
│ inflection position test mse │             0.0              │
│    inflection value loss     │    2.0386896721902303e-06    │
│  inflection value test mse   │    2.0386896721902303e-06    │
│      max position loss       │             0.0              │
│    max position test mse     │             0.0              │
│        max value loss        │    6.0647922509815544e-05    │
│     max value test loss      │    6.0647922509815544e-05    │
│       prediction loss        │    0.00018386635929346085    │
│     prediction test mse      │    0.00018386635929346085    │
│       test_loss_epoch        │    5.0284794269828126e-05    │
│      weighted mse loss       │     3.66678686987143e-05     │
│  weighted mse loss test mse  │     3.66678686987143e-05     │
└──────────────────────────────┴──────────────────────────────┘

 57%|█████▋    | 20/35 [24:55<07:25, 29.69s/it]

model: outputs/AE/AE_CNN_3D/dataset_output/8_channels_spatial_pooling_with_pca_pre_treatment/pred_0.01_grad_1000_max_pos_0.01_max_value_0.05_fft_0.0001_weighted_1_inflection_pos_0.01_inflection_value_0.05/depth_pre_treatment_pca_n_components_10/channels_[1, 8]/upsample_mode_trilinear/linear_later_False_lattent_size_50/1_conv_per_layer/padding_reflect/interp_size_25/pooling_Max_on_dim_spatial/final_upsample_upsample_pooling/act_fn_Elu_final_act_fn_Elu/lr_0.001/normalization_min_max/manage_nan_suppress/n_profiles_None/2024-11-13_15-43/checkpoints/val_loss=0.00-epoch=213.ckpt
